Implementation of the base model (June 12)
---

In [3]:
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd
import openpyxl

## 0 Instances

### First: Read the file from local directory

Arthur's:

'/Users/arthurdebelle/Desktop/TUM/SoSe 2024/Ad.S - OM/Project/CODING/Airports data/Brussels (EBBR)/Brussels_Clean.xlsm'

Ting-Ying's:

'/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm'

In [9]:
''' For now: change path of file here under'''
''' Later: put excels in same directory!!! '''

# Reading the number of flights Brussels airport
flightCount = len(pd.read_excel('/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm', sheet_name='EBBR - Flights', usecols='A:T', skiprows=2))
flightsBrussels = pd.read_excel('/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm', usecols='A:T', skiprows=1, nrows=flightCount)

# Reading the number of Gates from Brussels airport
gateCount = len(pd.read_excel('/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm', sheet_name='EBBR - Gates', usecols='A:D', skiprows=1))
gatesBrussels = pd.read_excel('/Users/chentingying/Documents/tum/Ad_Se_Operation_Management/Airports_data/Brussels_Clean.xlsm', sheet_name='EBBR - Gates', usecols='A:D', skiprows=0, nrows=gateCount)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [13]:
### Model 4 (CPP) ###
'''
# n = number of flights, m = number of gates
# T = matrix of time differences, P = preferences matrix
# alpha1, alpha2, alpha3 = weight factors, t_max = maximum buffer time
# U = successor function, M = valid gate assignments for each flight
# '''

# Inputs
n = flightCount # Number of flights
m = gateCount # Number of gates

# Flight columns
Flight_No = flightsBrussels['Flight']   # Aircrafts' flight number @ arrival
ETA = flightsBrussels['ETA']
ETD = flightsBrussels['ETD']
E_Parking = flightsBrussels['Planned Duration']
RTA = flightsBrussels['RTA']
RTD = flightsBrussels['RTD']
R_Parking = flightsBrussels['Real Duration']
Tot_Delay = flightsBrussels['Total Delay']
AC_size = flightsBrussels['AC size (m)']

# Tests
FN2 = Flight_No.tolist()

# Gates columns
Gate_Name = gatesBrussels['Name']
Max_Wingspan = gatesBrussels['Max length (m)']   # Max wingspan allowed on that gate

'''No idea what this is'''
# matrix of time differences
# buffer time
# T = [
#     [0, 10, 15, 20],  # Flight 1 can sequentially follow Flights 2, 3, 4
#     [10, 0, 15, 25],  # Flight 2 can sequentially follow Flights 1, 3, 4
#     [15, 15, 0, 30],  # Flight 3 can sequentially follow Flights 1, 2, 4
#     [20, 25, 30, 0]   # Flight 4 can sequentially follow Flights 1, 2, 3
# ]

# Building U -> U[i] = successor of i (=0 if no successor)
U = []
for i in range(n):
    U.extend([3*i+1, 3*i+2, 0])     # U = [1,2,0, 4,5,0, ...]
# print(f"U = {U}")

'''
i = 0,1,2 = "Landing", "Parking" and "Departure" index for the 1st flight of the day; i = 3,4,5 = ...
U[1] = 2    -> Parking of 1st flight has Departure of 1st flight as successor
U[5] = 0    -> Departure of 2nd flight has no successor
i%3 = 0     -> landing of a flight x,     with x = (i-i%3)/3 
i%3 = 1     -> parking of a flight x,     with x = (i-i%3)/3 
i%3 = 2     -> departure of a flight x,   with x = (i-i%3)/3 
'''

# Building M
M = Gate_Name.tolist()
'''
For now: M = [0,1,2,...] = allowed gates for ALL flights
Later: make it [[0, 3, 50, ...], ...] when we have preferences for each flights individually
'''

alpha1 = 1   # Preference scaling factor
alpha2 = 50  # Reward for avoiding tows
alpha3 = 5   # Penalty scaling factor for buffer time deficits

t_max = 30

# Shadow Restrictions
shadow_restrictions = [
    (0, 0, 1, 1), # Activity 1 at Gate 0 and Activity 2 at Gate 1 cannot happen
    (2, 2, 3, 1)  # Activity 3 at Gate 2 and Activity 4 at Gate 1 cannot happen
]

In [14]:
print(n)
print(m)

174
149


In [23]:
# Example inputs (For debugging)
n = 4  # Number of flights
m = 2  # Number of real gates

T = [
    [0, 10, 15, 20],  # Flight 1 can sequentially follow Flights 2, 3, 4
    [10, 0, 15, 25],  # Flight 2 can sequentially follow Flights 1, 3, 4
    [15, 15, 0, 30],  # Flight 3 can sequentially follow Flights 1, 2, 4
    [20, 25, 30, 0]   # Flight 4 can sequentially follow Flights 1, 2, 3
]
P = [
    [50, 30],  # Preferences of Flight 1 for Gates 1 and 2
    [40, 60],  # Preferences of Flight 2 for Gates 1 and 2
    [30, 70],  # Preferences of Flight 3 for Gates 1 and 2
    [20, 80]   # Preferences of Flight 4 for Gates 1 and 2
]
U = [0, 1, 2, 3]
M = [
    [0, 1],  # Flight 1 can go to Gates 1 and 2
    [0, 1],  # Flight 2 can go to Gates 1 and 2
    [0, 1],  # Flight 3 can go to Gates 1 and 2
    [0, 1]   # Flight 4 can go to Gates 1 and 2
]

alpha1 = 1   # Preference scaling factor
alpha2 = 50  # Reward for avoiding tows
alpha3 = 5   # Penalty scaling factor for buffer time deficits

t_max = 30

shadow_constraints = [
    (0, 1),  # Flights 1 and 2 cannot be in the same clique.
    (2, 3),  # Flights 3 and 4 cannot be in the same clique.
    (0, 5),  # Flight 1 and Gate 2 cannot be in the same clique. (assuming 5 is the index for Gate 2)
    (4, 5)   # Gate 1 and Gate 2 cannot be in the same clique. (assuming 4 is the index for Gate 1 and 5 is the index for Gate 2)
]

In [32]:
# Example inputs (Bigger)
n = 6  # Number of flights
m = 3  # Number of real gates

T = [
    [0, 10, -5, 20, 25, 30],  # Flight 1 can sequentially follow Flights 2, 4, 5, 6
    [10, 0, 15, 25, -10, 30],  # Flight 2 can sequentially follow Flights 1, 3, 4, 6
    [-5, 15, 0, 30, 35, 40],  # Flight 3 can sequentially follow Flights 2, 4, 5, 6
    [20, 25, 30, 0, 15, 20],  # Flight 4 can sequentially follow Flights 1, 2, 3, 5, 6
    [25, -10, 35, 15, 0, 30],  # Flight 5 can sequentially follow Flights 1, 2, 3, 4, 6
    [30, 30, 40, 20, 30, 0]   # Flight 6 can sequentially follow Flights 1, 2, 3, 4, 5
]
P = [
    [50, 30, 20],  # Preferences of Flight 1 for Gates 1, 2, and 3
    [40, 60, 20],  # Preferences of Flight 2 for Gates 1, 2, and 3
    [30, 70, 50],  # Preferences of Flight 3 for Gates 1, 2, and 3
    [20, 80, 60],  # Preferences of Flight 4 for Gates 1, 2, and 3
    [60, 40, 30],  # Preferences of Flight 5 for Gates 1, 2, and 3
    [70, 50, 40]   # Preferences of Flight 6 for Gates 1, 2, and 3
]
U = [0, 1, 2, 3, 4, 5]
M = [
    [0, 1, 2],  # Flight 1 can go to Gates 1, 2, and 3
    [0, 1, 2],  # Flight 2 can go to Gates 1, 2, and 3
    [0, 1, 2],  # Flight 3 can go to Gates 1, 2, and 3
    [0, 1, 2],  # Flight 4 can go to Gates 1, 2, and 3
    [0, 1, 2],  # Flight 5 can go to Gates 1, 2, and 3
    [0, 1, 2]   # Flight 6 can go to Gates 1, 2, and 3
]

alpha1 = 1   # Preference scaling factor
alpha2 = 50  # Reward for avoiding tows
alpha3 = 5   # Penalty scaling factor for buffer time deficits

t_max = 30

# Example shadow constraints based on hypothetical rules
shadow_constraints = [
    (0, 1),  # Flight 1 and Flight 2
    (2, 3),  # Flight 3 and Flight 4
    (1, 5),  # Flight 2 and Gate 2 (assuming 5 is the index for Gate 2)
    (6, 7),  # Gate 1 and Gate 2 (assuming 6 is the index for Gate 1 and 7 is the index for Gate 2)
    (4, 8),  # Flight 5 and Gate 3 (assuming 8 is the index for Gate 3)
    (7, 8)   # Gate 2 and Gate 3
]


1 Flight-Gate Scheduling (FGS) Problem
---

In [7]:
def build_FGS_model(n, m, P, U, T, M, shadow_restrictions, alpha1, alpha2, alpha3, t_max):
    model = Model("FlightGateScheduling")

    # Decision variables for each activity being assigned to each gate
    x = model.addVars(n, m + 1, vtype=GRB.BINARY, name="x") # Number of real gates is num_gates; the last index is assumed to be the dummy gate

    # Constraints
    # Assign each activity to exactly one of its allowable gates
    for i in range(n):
        model.addConstr(quicksum(x[i, j] for j in range(m) if j in M[i]) == 1, name=f"Assign_{i}") 
        '''can include dummy gate it in this quicksum (might leader to a tighter LP relaxation)'''

    # Non-overlapping constraint (1)
    for i in range(n):
        for j in range(n):
            if T[i][j] < 0:
                for k in range(m):
                    model.addConstr(x[i, k] + x[j, k] <= 1, name=f"Overlap_{i}_{j}_{k}")

    # Shadow restrictions (2)
    for (i, k, j, l) in shadow_restrictions:
        model.addConstr(x[i, k] + x[j, l] <= 1, name=f"Shadow_{i}_{j}")

    # Objective components (3)
    
    # Append significantly negative scores for the dummy gate
    P_star = [prefs + [-1000] for prefs in P]  # Add a very undesirable score for the dummy gate

    # z1: Minimize the negative sum of adjusted preferences
    z1 = quicksum(P_star[i][j] * x[i, j] for i in range(n) for j in range(m + 1)) 
    ''' - quicksum(P_star[i][j] * x[i, j] for (i,j) in x)
    negative is important!
    '''
    
   
    # Define a new decision variable for tows between activities and their successors
    '''Good practice: first define ALL variables, then define ALL constraints, and ultimately define the objective function
    '''
    tows = model.addVars(n, m, vtype=GRB.BINARY, name="tows")
    '''Towing to the dummy gate also needs to be possible (-> tighter lower bounds)'''

    # Update constraints to calculate tows:
    for i in range(n):
        '''Each activity must have a successor, i.e. U[i] \in N for all i
        In the trivial case that would be the activity itself (->U[i] = i)'''
        if U[i] != 0:  # Has a valid successor 
            for k in range(m):
                # If activity i is assigned to gate k and its successor U[i] to a different gate
                model.addConstr(tows[i, k] >= x[i, k] - x[U[i], k], name=f"TowIfDifferent1_{i}_{k}")
                model.addConstr(tows[i, k] >= x[U[i], k] - x[i, k], name=f"TowIfDifferent2_{i}_{k}")
                '''If I understand it correctly, it should be that tows[i,k] = 1 <=> flight associated with activity i
                needs to be towed away from gate k after activity i is done.
                First constraint: "if activity i is assigned to gate k, but its successor is not, we need to tow the plane 
                away from k after finishing i"
                second constraint: "if activity i is NOT assigned to gate k, but its success is, we need to tow the plane
                towards k after finishing i"
                The latter constraint set would then lead to every tow being counted twice if I'm not mistaken
                '''

    # Redefine z2 using the new tow variables
    z2 = quicksum(tows[i, k] for i in range(n) for k in range(m))

    # z3: Buffer time deficit
    z3 = quicksum(max(t_max - T[i][j], 0) * x[i, k] * x[j, k] 
                  for i in range(n) for j in range(i +1, n) for k in range(m))
    '''This is a quadratic constraint. It can be linearized by introducing variables buffer[i,j] for activities i and j,
    bounding them from above by x[i,k]*(max(t_max - T_[i][j],0)) and x[j,k]*(max(t_max - T_[i][j],0)).
    '''
    
    
    # Combined objective
    model.setObjective(alpha1 * z1 + alpha2 * z2 + alpha3 * z3, GRB.MINIMIZE)
    model.optimize()

    return model, x, tows

<>:44: SyntaxWarning: invalid escape sequence '\i'
<>:44: SyntaxWarning: invalid escape sequence '\i'
/var/folders/mw/4d26229d25302_j476mymn740000gn/T/ipykernel_7437/2471310534.py:44: SyntaxWarning: invalid escape sequence '\i'
  '''Each activity must have a successor, i.e. U[i] \in N for all i


In [8]:
for idx in x:
    if x[idx].X > 0.5:
        print(idx)

NameError: name 'x' is not defined

In [ ]:
# Build the model
model, x = build_FGS_model(n, m, P, U, T, M, shadow_restrictions, alpha1, alpha2, alpha3, t_max)  # Receive model and decision variables

# Print the results
if model.status == GRB.OPTIMAL:
    assignments = model.getAttr('X', x)
    for i in range(n):
        for j in range(m):
            if assignments[i, j] > 0.5:
                print(f"Activity {i} assigned to Gate {j}")


2 FGS as Clique Partitioning Problem (CPP)
---

Assumptions
1. Flights are represented as nodes in a graph.
2. Conflicts between flights are represented as edges.
3. Each clique represents a set of flights that can be assigned to the same gate without conflict.

In [33]:
# Callback function to add lazy constraints
def mycallback(model, where):
    if where == GRB.Callback.MIPSOL:
        for (i, j) in model._shadow_constraints:
            if (i, j) in model._x and model.cbGetSolution(model._x[i, j]) > 0.5:
                model.cbLazy(model._x[i, j] == 0)

In [49]:
def build_cpp_model(n, m, T, P, U, M, alpha1, alpha2, alpha3, t_max):
    model = Model("GateAssignmentCPP")
    model.setParam(GRB.Param.LazyConstraints, 1)  # Enable lazy constraints

    # Total vertices: flights + gates (5)
    a = n + m - 1
    
    # Initialize the edge weights matrix
    weights = [[0] * a for _ in range(a)]
    '''weights = [0]*len(a)''' #But "a" in an int, and object of type 'int' has no len()
    large_negative = -1e6  # A large negative number to represent -inf

    # Populate the weights matrix based on given rules (6)
    for i in range(n):
        for j in range(n):
            if i != j:
                if T[i][j] < 0: # Activities overlap in time
                    weights[i][j] = large_negative
                    
                    '''What will be a recommanded way to handle -inf? 
                    -inf is not allowed in weight matrix
                    Because I can't multiply and sum them in the objective function'''

                elif U[i] == j or U[j] == i: # Saving a tow
                    weights[i][j] = alpha2  
                else:
                    weights[i][j] = -alpha3 * max(t_max - T[i][j], 0)  # Buffer time difference
                
    # Weights for flight to gate assignments (7)
    for i in range(n):
        for j in range(n, a):
            if j - n in M[i]:
                weights[i][j] = alpha1 * P[i][j - n]
            else:
                weights[i][j] = large_negative
    
    # Gates cannot be in the same clique (8)
    for i in range(n, a):
        for j in range(n, a):
            weights[i][j] = large_negative

    # Debugging: Print weights matrix to check for NaN or Inf values
    for row in weights:
        print(row)
    
    # Decision variables: x[i, j] = 1 if i and j are in the same clique, 0 otherwise
    '''Your variable space is twice as large as it needs to be. Instead, only define variables x[i,j] for i < j.'''
    '''How about the current one?'''
    x = {}
    for i in range(a):
        for j in range(i + 1, a):
            x[i, j] = model.addVar(vtype=GRB.BINARY, name=f"x[{i},{j}]")
    
    # Objective: Maximize the sum of the weights for edges within the same clique
    model.setObjective(quicksum(weights[i][j] * x[i, j] for i in range(a) for j in range(i + 1, a)), GRB.MAXIMIZE)

    # Add transitivity constraints for forming valid cliques (4)
    for i in range(a):
        for j in range(a):
            for k in range(a):
                #''' 1 <= i and k <= a are not needed. Your iterator for i starts at 0 and k <= a holds by definition.'''
                if i < j and j < k:
                    model.addConstr(x[i, j] + x[j, k] - x[i, k] <= 1)
                    model.addConstr(x[i, j] - x[j, k] + x[i, k] <= 1)
                    model.addConstr(-x[i, j] + x[j, k] + x[i, k] <= 1)

    # Store variables and constraints for the callback
    model._x = x
    model._shadow_constraints = shadow_constraints

    # Optimize with the callback function to add lazy constraints
    model.optimize(mycallback)
    
    return model, x

In [50]:
# Build the model
model, x = build_cpp_model(n, m, T, P, U, M, alpha1, alpha2, alpha3, t_max)

# Print solution
if model.status == GRB.Status.OPTIMAL:
    print(f"Optimal solution found with total score: {model.objVal}")
    solution = model.getAttr('x', x)
    cliques = {}
    for (i, j), value in solution.items():
        if value > 0.5:
            if i in cliques:
                cliques[i].append(j)
            else:
                cliques[i] = [j]
            if j in cliques:
                cliques[j].append(i)
            else:
                cliques[j] = [i]

    # Print cliques
    for i in range(n):
        if i in cliques:
            clique = [i] + cliques[i]
            flights = [f"Flight {x+1}" for x in clique if x < n]
            gates = [f"Gate {x-n+1}" for x in clique if x >= n]
            print(f"Flight {i+1} is in a clique with {', '.join(flights + gates)}")
        else:
            print(f"Flight {i+1} did not obtain any particular gate assignment")

    for i in range(n, n + m):
        if i in cliques:
            clique = [i] + cliques[i]
            gates = [f"Gate {x-n+1}" for x in clique if x >= n]
            flights = [f"Flight {x+1}" for x in clique if x < n]
            print(f"Gate {i-n+1} is in a clique with {', '.join(gates + flights)}")
else:
    print("No optimal solution found.")

Set parameter LazyConstraints to value 1
[0, -100, -1000000.0, -50, -25, 0, 50, 30]
[-100, 0, -75, -25, -1000000.0, 0, 40, 60]
[-1000000.0, -75, 0, 0, 0, 0, 30, 70]
[-50, -25, 0, 0, -75, -50, 20, 80]
[-25, -1000000.0, 0, -75, 0, 0, 60, 40]
[0, 0, 0, -50, 0, 0, 70, 50]
[0, 0, 0, 0, 0, 0, -1000000.0, -1000000.0]
[0, 0, 0, 0, 0, 0, -1000000.0, -1000000.0]
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 168 rows, 28 columns and 504 nonzeros
Model fingerprint: 0xea985a59
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 110.0000000
Presolve time: 0.00s
Presolved: 168 rows, 28 columns, 504 nonzeros
Variable types: 0 continuous, 28 integer (28

## Draw a graph

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_cpp_graph(flights, gates, connections):
    """
    Draws a CPP graph with given flight and gate nodes and their connections in a hexagonal-like layout.
    
    Parameters:
    - flights (dict): Dictionary where keys are flight labels and values are dicts of gate connections with weights.
    - gates (list): List of gate labels.
    - connections (dict): Optional additional edges between flights with weights.
    """
    # Create a graph
    G = nx.Graph()

    # Add nodes for flights and gates
    G.add_nodes_from(flights.keys(), style='filled', fillcolor='red', shape='circle')
    G.add_nodes_from(gates, style='filled', fillcolor='blue', shape='square')

    # Add edges between flights and gates with preferences
    for flight, prefs in flights.items():
        for gate, weight in prefs.items():
            G.add_edge(flight, gate, weight=weight)

    # Add optional additional connections between flights
    for (f1, f2), weight in connections.items():
        G.add_edge(f1, f2, weight=weight)

    # Manually define positions for a hexagonal-like layout
    pos = {
        'F1': (-1, 1), 'F2': (0, 2), 'F3': (1, 1),  # Top vertex, and middle row vertices
        'F4': (0, 0),  # Bottom vertex of the upper hexagon (if we imagine a hexagon)
        'G1': (-0.5, -1), 'G2': (0.5, -1)  # Bottom vertices (gates)
    }

    # Draw the network
    nx.draw(G, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=2000, font_size=15, font_weight='bold')

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

    plt.title("Clique Partitioning Problem Graph - Hexagonal Layout")
    plt.axis('off')  # Hide the axes
    plt.show()

# Example usage
flights = {
    'F1': {'G1': 50, 'G2': 30},
    'F2': {'G1': 40, 'G2': 60},
    'F3': {'G1': 30, 'G2': 70},
    'F4': {'G1': 20, 'G2': 80}
}
gates = ['G1', 'G2']
additional_connections = {('F1', 'F2'): 10, ('F2', 'F3'): 15}

draw_cpp_graph(flights, gates, additional_connections)
